# 1번
CNN 모형에서 mnist데이터의  pixel 의 size 가 maxpooling 에 의해 줄어들 때, channel 수를 maxpooling 의 kernel size 만큼 증가시켜 원래 모형과 비교하라.

### mnist 데이터 불러오고 reshape 하기

In [7]:
import numpy as np
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

In [8]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
num_labels=len(np.unique(y_train))
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

shape1=x_train.shape[1]
shape2=x_train.shape[2]

#cnn 에 쓸 수 있도록 4D tensor 로 reshape 한다.
x_train=x_train.reshape(-1, shape1,shape2,1).astype('float32')
x_test=x_test.reshape(-1, shape1,shape2,1).astype('float32')
x_train=x_train/255.
x_test=x_test/255.

#표본수를 포함하여 4D 텐서이므로 입력은 3D 텐서가 된다.
input_shape=(shape1,shape2,1)
batch_size=64 # batch size를 정해놓음
kernel_size=3
pool_size=2
filters=64
dropout=0.3

### CNN 모델 생성 - Sequential API|

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

**기존 모형) kernel size = 2 인 max_pooling 을 적용함**

In [4]:
model=Sequential()
model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=input_shape))
# kernel size = 2인 max_pooling
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu'))
# kernel size = 2인 max_pooling
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu'))
model.add(Flatten())
model.add(Dropout(dropout))
model.add(Dense(num_labels, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dropout (Dropout)            (None, 576)               0

In [5]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
938/938 [==============================] - 56s 58ms/step - loss: 0.4675 - accuracy: 0.85211s - loss: 0.4738 - accuracy:  - ETA: 
Epoch 2/10
938/938 [==============================] - 57s 61ms/step - loss: 0.0655 - accuracy: 0.9786
Epoch 3/10
938/938 [==============================] - 58s 62ms/step - loss: 0.0469 - accuracy: 0.9852
Epoch 4/10
938/938 [==============================] - 56s 60ms/step - loss: 0.0356 - accuracy: 0.9883
Epoch 5/10
938/938 [==============================] - 60s 64ms/step - loss: 0.0289 - accuracy: 0.9909
Epoch 6/10
938/938 [==============================] - 59s 62ms/step - loss: 0.0257 - accuracy: 0.9920
Epoch 7/10
938/938 [==============================] - 62s 67ms/step - loss: 0.0199 - accuracy: 0.9931
Epoch 8/10
938/938 [==============================] - 66s 70ms/step - loss: 0.0177 - accuracy: 0.9937
Epoch 9/10
938/938 [==============================] - 64s 68ms/step - loss: 0.0166 - accuracy: 0.9950
Epoch 10/10
938/938 [=======================

In [6]:
results=model.evaluate(x_test,y_test, batch_size=128)
print(results)

79/79 [==============================] - 3s 31ms/step - loss: 0.0239 - accuracy: 0.9925  - ETA: 0s - loss: 0.0242 - accuracy: 0.
[0.023880712687969208, 0.9925000071525574]


**새로운 모형) maxpooling 이후 1X1 convolution layer를 추가해 channel 수를 늘림**

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

new_model=Sequential()
new_model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=input_shape))
new_model.add(MaxPooling2D(pool_size)) # kernel size = 2인 max_pooling
new_model.add(Conv2D(filters=filters*pool_size,kernel_size=1)) # channel 수를 pool_size 만큼 늘림
new_model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu'))
new_model.add(MaxPooling2D(pool_size)) # kernel size = 2인 max_pooling
new_model.add(Conv2D(filters=filters*pool_size,kernel_size=1)) # channel 수를 pool_size 만큼 늘림
new_model.add(Conv2D(filters=filters,kernel_size=kernel_size,activation='relu'))
new_model.add(Flatten())
new_model.add(Dropout(dropout))
new_model.add(Dense(num_labels, activation='softmax'))
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 128)       8320      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 5, 5, 128)         8320      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 3, 64)         

In [12]:
new_model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
new_model.fit(x_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
938/938 [==============================] - 94s 100ms/step - loss: 0.3859 - accuracy: 0.8742
Epoch 2/10
938/938 [==============================] - 96s 102ms/step - loss: 0.0611 - accuracy: 0.9815
Epoch 3/10
938/938 [==============================] - 101s 108ms/step - loss: 0.0429 - accuracy: 0.9869
Epoch 4/10
938/938 [==============================] - 97s 104ms/step - loss: 0.0362 - accuracy: 0.9887
Epoch 5/10
938/938 [==============================] - 92s 98ms/step - loss: 0.0313 - accuracy: 0.9904
Epoch 6/10
938/938 [==============================] - 88s 94ms/step - loss: 0.0284 - accuracy: 0.9913
Epoch 7/10
938/938 [==============================] - 85s 91ms/step - loss: 0.0247 - accuracy: 0.9917
Epoch 8/10
938/938 [==============================] - 88s 94ms/step - loss: 0.0230 - accuracy: 0.9930
Epoch 9/10
938/938 [==============================] - 87s 93ms/step - loss: 0.0202 - accuracy: 0.9933
Epoch 10/10
938/938 [==============================] - 86s 92ms/step - loss: 

In [14]:
results=new_model.evaluate(x_test,y_test, batch_size=128)
print(results)

79/79 [==============================] - 3s 39ms/step - loss: 0.0254 - accuracy: 0.9926
[0.025436801835894585, 0.9926000237464905]


# 2번

(1) Boston-housing 데이터를 tensorflow dataset API 에서 내려받아, traing data 와 test data 를 구성하라.

(2) Boston-housing 데이터를 그림 4-1 아키텍쳐에 적용한 후, 모형의 최적화를 callbacks 로 구현하라.

### Boston Housing data 불러오기

In [15]:
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing

(x_train,y_train),(x_test,y_test) = boston_housing.load_data()
print(x_train.dtype,y_train.dtype)
print(x_train.shape,y_train.shape)
print(x_test.shape)

float64 float64
(404, 13) (404,)
(102, 13)


input data를 각각 10개의 특성변수를 갖도록 하며 2개로 나눔 

In [ ]:
x_train1, x_train2 = x_train[:,:10], x_train[:,3:]
x_test1, x_test2 =  x_test[:,:10], x_test[:,3:]

### 그림 4-1 아키텍처와 같은 모델 생성 - Functional API

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,concatenate
from tensorflow.keras.utils import plot_model

input_A=Input(shape=[10])
input_B=Input(shape=[10])
h1=Dense(16,activation='relu')(input_A) #그림 4-1에서 dense1에 해당
h2=Dense(16,activation='relu')(h1) #dense2에 해당
concat=concatenate([input_B,h2]) #inputB와 h2를 연결. concatenate.
out1=Dense(1)(h2) #output 하나를 꺼내고
out2=Dense(1)(concat) #concat 해서 output2를 만듦
model=Model(inputs=[input_A,input_B],outputs=[out1,out2])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           176         input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 16)           272         dense_2[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10)]         0                                            
______________________________________________________________________________________________

### callback 함수로 EarlyStopping 과 ModelCheckpoint 를 사용
compile & fit
* epoch를 200으로 충분히 크게 하고 earlystopping을 patience=10으로 적용하였다.
* val_loss 를 earlystopping의 기준으로 설정하였다.
* ModelCheckpoint를 통해 val_loss 가 가장 작은 model을 저장하였다.
* batch size는 64로 하였다.

In [4]:
#callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
callback_list = [EarlyStopping(monitor='val_loss',mode='min',patience=10),
                 ModelCheckpoint(filepath='model_boston_1.h5',monitor='val_loss',save_best_only='True'),
                TensorBoard(log_dir="./logs-1")]

#compile model
model.compile(loss=['mse','mse'],loss_weights=[0.2,0.8], optimizer='RMSprop')
#fit model
history = model.fit([x_train1,x_train2],[y_train,y_train],epochs=200, batch_size=64, callbacks=callback_list,
                    validation_data=([x_test1,x_test2],[y_test,y_test]))

Epoch 1/200
7/7 [==============================] - 5s 566ms/step - loss: 54058.5347 - dense_2_loss: 27001.9387 - dense_3_loss: 60822.6880 - val_loss: 39906.5742 - val_dense_2_loss: 17924.1836 - val_dense_3_loss: 45402.1680
Epoch 2/200
7/7 [==============================] - 0s 12ms/step - loss: 35181.1694 - dense_2_loss: 15712.7874 - dense_3_loss: 40048.2651 - val_loss: 27944.5000 - val_dense_2_loss: 11421.1396 - val_dense_3_loss: 32075.3398
Epoch 3/200
7/7 [==============================] - 0s 13ms/step - loss: 24698.9937 - dense_2_loss: 9875.8508 - dense_3_loss: 28404.7803 - val_loss: 18929.1270 - val_dense_2_loss: 6739.5762 - val_dense_3_loss: 21976.5156
Epoch 4/200
7/7 [==============================] - 0s 14ms/step - loss: 16802.3304 - dense_2_loss: 5923.9451 - dense_3_loss: 19521.9265 - val_loss: 12288.4170 - val_dense_2_loss: 3670.1807 - val_dense_3_loss: 14442.9766
Epoch 5/200
7/7 [==============================] - 0s 12ms/step - loss: 10686.8329 - dense_2_loss: 3166.9863 - dens

Epoch 40/200
7/7 [==============================] - 0s 14ms/step - loss: 77.7320 - dense_2_loss: 97.7393 - dense_3_loss: 72.7302 - val_loss: 106.3453 - val_dense_2_loss: 113.2621 - val_dense_3_loss: 104.6161
Epoch 41/200
7/7 [==============================] - 0s 19ms/step - loss: 81.0196 - dense_2_loss: 94.6361 - dense_3_loss: 77.6155 - val_loss: 76.3633 - val_dense_2_loss: 103.4674 - val_dense_3_loss: 69.5872
Epoch 42/200
7/7 [==============================] - 0s 16ms/step - loss: 76.4444 - dense_2_loss: 99.1216 - dense_3_loss: 70.7751 - val_loss: 76.5330 - val_dense_2_loss: 109.1047 - val_dense_3_loss: 68.3901
Epoch 43/200
7/7 [==============================] - 0s 13ms/step - loss: 85.9739 - dense_2_loss: 104.7767 - dense_3_loss: 81.2732 - val_loss: 75.4894 - val_dense_2_loss: 98.6287 - val_dense_3_loss: 69.7045
Epoch 44/200
7/7 [==============================] - 0s 13ms/step - loss: 70.7114 - dense_2_loss: 90.4130 - dense_3_loss: 65.7860 - val_loss: 85.7641 - val_dense_2_loss: 97.40

### Tensorboard로 결과 살펴보기

In [7]:
%load_ext tensorboard
%tensorboard --logdir {"./logs-1"} --port=8000

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 32412.

cf) callback 함수로 ReduceLROnPlateau 를 사용할 수도 있다.

In [18]:
#callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
callback_list = [ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)]

#compile model
model.compile(loss=['mse','mse'],loss_weights=[0.2,0.8], optimizer='RMSprop')
#fit model
history = model.fit([x_train1,x_train2],[y_train,y_train],epochs=70, batch_size=32, callbacks=callback_list,
                    validation_data=([x_test1,x_test2],[y_test,y_test]))


Epoch 1/70
13/13 [==============================] - 1s 33ms/step - loss: 4556.0460 - dense_4_loss: 15506.7233 - dense_5_loss: 1818.3764 - val_loss: 2269.6091 - val_dense_4_loss: 6755.5454 - val_dense_5_loss: 1148.1249
Epoch 2/70
13/13 [==============================] - 0s 7ms/step - loss: 2142.3176 - dense_4_loss: 5148.4999 - dense_5_loss: 1390.7719 - val_loss: 1224.7067 - val_dense_4_loss: 2414.9353 - val_dense_5_loss: 927.1495
Epoch 3/70
13/13 [==============================] - 0s 8ms/step - loss: 1243.5795 - dense_4_loss: 1849.3603 - dense_5_loss: 1092.1342 - val_loss: 777.9865 - val_dense_4_loss: 787.5747 - val_dense_5_loss: 775.5894
Epoch 4/70
13/13 [==============================] - 0s 6ms/step - loss: 844.6578 - dense_4_loss: 634.2230 - dense_5_loss: 897.2664 - val_loss: 592.0190 - val_dense_4_loss: 373.5371 - val_dense_5_loss: 646.6395
Epoch 5/70
13/13 [==============================] - 0s 7ms/step - loss: 750.7629 - dense_4_loss: 341.4003 - dense_5_loss: 853.1036 - val_loss: 4

13/13 [==============================] - 0s 6ms/step - loss: 80.8814 - dense_4_loss: 87.3127 - dense_5_loss: 79.2736 - val_loss: 94.7990 - val_dense_4_loss: 104.7781 - val_dense_5_loss: 92.3042
Epoch 41/70
13/13 [==============================] - 0s 7ms/step - loss: 73.2181 - dense_4_loss: 83.7930 - dense_5_loss: 70.5744 - val_loss: 105.8288 - val_dense_4_loss: 109.5689 - val_dense_5_loss: 104.8938
Epoch 42/70
13/13 [==============================] - 0s 6ms/step - loss: 86.6369 - dense_4_loss: 91.0326 - dense_5_loss: 85.5380 - val_loss: 97.3136 - val_dense_4_loss: 102.7119 - val_dense_5_loss: 95.9640
Epoch 43/70
13/13 [==============================] - 0s 7ms/step - loss: 81.2271 - dense_4_loss: 92.8929 - dense_5_loss: 78.3106 - val_loss: 91.8076 - val_dense_4_loss: 104.4358 - val_dense_5_loss: 88.6505
Epoch 44/70
13/13 [==============================] - 0s 7ms/step - loss: 85.9435 - dense_4_loss: 100.3410 - dense_5_loss: 82.3441 - val_loss: 107.6786 - val_dense_4_loss: 110.9389 - val_